In [0]:
dbutils.fs.ls("dbfs:/FileStore/tables/")


Out[4]: [FileInfo(path='dbfs:/FileStore/tables/cell_phone.xlsx', name='cell_phone.xlsx', size=297905, modificationTime=1740663046000),
 FileInfo(path='dbfs:/FileStore/tables/cell_phones-1.csv', name='cell_phones-1.csv', size=43927, modificationTime=1740667679000),
 FileInfo(path='dbfs:/FileStore/tables/cell_phones-2.csv', name='cell_phones-2.csv', size=43927, modificationTime=1740667926000),
 FileInfo(path='dbfs:/FileStore/tables/cell_phones-3.csv', name='cell_phones-3.csv', size=43927, modificationTime=1740717554000),
 FileInfo(path='dbfs:/FileStore/tables/cell_phones-4.csv', name='cell_phones-4.csv', size=43927, modificationTime=1740851516000),
 FileInfo(path='dbfs:/FileStore/tables/cell_phones.csv', name='cell_phones.csv', size=43927, modificationTime=1740650761000),
 FileInfo(path='dbfs:/FileStore/tables/cleaned_ipl_2021/', name='cleaned_ipl_2021/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/FileStore/tables/final_cleaned_ipl_2021_matches.csv/', name='final_cleaned_ipl_2021

In [0]:

ipl_matches_df = spark.read.option("header", "true").csv("dbfs:/FileStore/tables/ipl_2021_matches.csv")


In [0]:
ipl_matches_df = spark.read.option("header", "true").csv("dbfs:/FileStore/tables/ipl_2021_matches.csv")
ipl_matches_df.show(5, truncate=False)


+----+-----------+--------------------------+--------+----------+--------------------------------------------------------+---------------------------------------------------------------------------------------------+-----------+-----------+--------------------+--------------+-----------+
|year|series_type|series_name               |match_no|match_type|match_name                                              |match_href                                                                                   |match_team1|match_team2|match_datetime_start|match_date_end|match_venue|
+----+-----------+--------------------------+--------+----------+--------------------------------------------------------+---------------------------------------------------------------------------------------------+-----------+-----------+--------------------+--------------+-----------+
|2021|T20        |Indian Premier League 2021|null    |League    |MUMBAI INDIANS vs ROYAL CHALLENGERS BENGALURU, 1st Match|https://www

In [0]:
from pyspark.sql.functions import lit

In [0]:
ipl_matches_df = spark.read.option("header", "true").csv("dbfs:/FileStore/tables/ipl_2021_matches.csv")

In [0]:
from pyspark.sql import Row

commentary_data = [
    Row(match_id=1, over_number=1, ball_number=1, commentary="First ball of the match!"),
    Row(match_id=1, over_number=1, ball_number=2, commentary="The batsman takes a single."),
    Row(match_id=1, over_number=1, ball_number=3, commentary="Good length delivery, defended."),
    Row(match_id=2, over_number=2, ball_number=1, commentary="New over, fresh batsman at the crease."),
    Row(match_id=2, over_number=2, ball_number=2, commentary="Shot! Boundary to start the over."),
    Row(match_id=3, over_number=3, ball_number=1, commentary="Change of pace from the bowler."),
    Row(match_id=3, over_number=3, ball_number=2, commentary="The batsman drives it down the ground.")
]


In [0]:
commentary_df = spark.createDataFrame(commentary_data)

In [0]:
match_data = [
    Row(match_id=1, match_name="MI vs RCB, 1st Match", match_date="2021-04-09", match_time="19:30:00"),
    Row(match_id=2, match_name="CSK vs DC, 2nd Match", match_date="2021-04-10", match_time="19:30:00"),
    Row(match_id=3, match_name="SRH vs KKR, 3rd Match", match_date="2021-04-11", match_time="20:00:00")
]

In [0]:
match_data_df = spark.createDataFrame(match_data)

In [0]:
commentary_df = commentary_df.withColumn("match_name", lit(None).cast("string")) \
                              .withColumn("match_date", lit(None).cast("string")) \
                              .withColumn("match_time", lit(None).cast("string"))

match_data_df = match_data_df.withColumn("over_number", lit(None).cast("int")) \
                              .withColumn("ball_number", lit(None).cast("int")) \
                              .withColumn("commentary", lit(None).cast("string"))


In [0]:
combined_df = commentary_df.unionByName(match_data_df)

In [0]:
combined_df.show(20, truncate=False)

+--------+-----------+-----------+--------------------------------------+---------------------+----------+----------+
|match_id|over_number|ball_number|commentary                            |match_name           |match_date|match_time|
+--------+-----------+-----------+--------------------------------------+---------------------+----------+----------+
|1       |1          |1          |First ball of the match!              |null                 |null      |null      |
|1       |1          |2          |The batsman takes a single.           |null                 |null      |null      |
|1       |1          |3          |Good length delivery, defended.       |null                 |null      |null      |
|2       |2          |1          |New over, fresh batsman at the crease.|null                 |null      |null      |
|2       |2          |2          |Shot! Boundary to start the over.     |null                 |null      |null      |
|3       |3          |1          |Change of pace from th

In [0]:
combined_df.write.mode("overwrite").option("header", "true").csv("dbfs:/FileStore/tables/combined_ipl_commentary.csv")


In [0]:
final_combined_df = combined_df.filter(
    (F.col("over").isNotNull()) | (F.col("ball_type").isNotNull()) | (F.col("commentary").isNotNull()) | (F.col("team").isNotNull())
)
final_combined_df.show(truncate=False)

+----+---------+----------------------------------------+------+
|over|ball_type|commentary                              |team  |
+----+---------+----------------------------------------+------+
|1.1 |normal   |First ball of the match! Player A faces.|Team A|
|1.2 |wide     |The batsman takes a quick single.       |Team B|
+----+---------+----------------------------------------+------+

